<a href="https://colab.research.google.com/github/isamdr86/towards-ai/blob/main/notebooks/Advanced_Retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages and Setup Variables


In [1]:
!pip install -q llama-index==0.10.49 llama-index-vector-stores-chroma==0.1.9 llama-index-llms-gemini==0.1.11 google-generativeai==0.5.4 openai==1.35.3 chromadb==0.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.1/679.1 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.5 MB/s eta 0

In [2]:
%%capture
!pip install openai==1.55.3 httpx==0.27.2 tiktoken==0.7.0 --force-reinstall --quiet

In [3]:
import os

import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')
os.environ["GOOGLE_API_KEY"] = userdata.get('google_api_key')


## Language Model and Embedding Model

In [4]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.gemini import Gemini
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI

# Settings.llm = OpenAI(temperature=0, model="gpt-4o-mini")

Settings.llm = Gemini(model="models/gemini-1.5-flash", temperature=1, max_tokens=512)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

**Note: You can create a vector store from scratch using the code below, or you can load it from Hugging Face using the code provided in this notebook.**

# Create a Vector Store


In [5]:
import chromadb

# create client and a new collection
# chromadb.EphemeralClient saves data in-memory.
chroma_client = chromadb.PersistentClient(path="./ai_tutor_knowledge")
chroma_collection = chroma_client.create_collection("ai_tutor_knowledge")

In [6]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.storage.storage_context import StorageContext

# Define a storage context object using the created vector database.
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Load the Dataset (JSON)


## Download


In [7]:
from huggingface_hub import hf_hub_download
file_path = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="ai_tutor_knowledge.jsonl",repo_type="dataset",local_dir="/content")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ai_tutor_knowledge.jsonl:   0%|          | 0.00/6.96M [00:00<?, ?B/s]

# Read File


In [8]:
import json
with open(file_path, "r") as file:
    ai_tutor_knowledge = [json.loads(line) for line in file]

len(ai_tutor_knowledge)

762

# Convert to Document obj


In [9]:
from typing import List
from llama_index.core import Document

def create_docs_from_list(data_list: List[dict]) -> List[Document]:
    documents = []
    for data in data_list:
        documents.append(
            Document(
                doc_id=data["doc_id"],
                text=data["content"],
                metadata={  # type: ignore
                    "url": data["url"],
                    "title": data["name"],
                    "tokens": data["tokens"],
                    "source": data["source"],
                },
                excluded_llm_metadata_keys=[
                    "title",
                    "tokens",
                    "source",
                ],
                excluded_embed_metadata_keys=[
                    "url",
                    "tokens",
                    "source",
                ],
            )
        )
    return documents

doc = create_docs_from_list(ai_tutor_knowledge)

# Transforming


In [10]:
from llama_index.core.node_parser import SentenceWindowNodeParser

# create the sentence window node parser
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    include_metadata=True,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

https://docs.llamaindex.ai/en/v0.10.19/api/llama_index.core.schema.TextNode.html

In [11]:
documents = [i for i in doc if i.metadata['tokens']<8000]
nodes = node_parser.get_nodes_from_documents(documents)

In [12]:
len(nodes)

22083

In [13]:
nodes[0]

TextNode(id_='7dff102f-7157-4518-b6dc-0de8f07b9a26', embedding=None, metadata={'window': 'Github Repo : https://github.com/vaibhawkhemka/ML-Umbrella/tree/main/MLops/Model_Deployment/Bert_Kubernetes_deployment   Model development is useless if you dont deploy it to production  which comes with a lot of issues of scalability and portability.    I have deployed a basic BERT model from the huggingface transformer on Kubernetes with the help of docker  which will give a feel of how to deploy and manage pods on production.    Model Serving and Deployment:ML Pipeline:Workflow:   Model server (using FastAPI  uvicorn) for BERT uncased model    Containerize model and inference scripts to create a docker image    Kubernetes deployment for these model servers (for scalability)  Testing   Components:Model serverUsed BERT uncased model from hugging face for prediction of next word [MASK].  Inference is done using transformer-cli which uses fastapi and uvicorn to serve the model endpoints   Server st

## Index creation

In [14]:
from llama_index.core import VectorStoreIndex

# Add the documents to the database and create Index / embeddings
index = VectorStoreIndex(nodes, storage_context=storage_context,show_progress =True)

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Compress the vector store directory to a zip file to be able to download and use later.
!zip -r vectorstore-windowed.zip ai_tutor_knowledge

# Load Indexes


**Note: If you created the vector store from scratch, please uncomment out the three code blocks/cells below.**

In [16]:
from huggingface_hub import hf_hub_download
vectorstore = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="vectorstore-windowed.zip",repo_type="dataset",local_dir="/content")

vectorstore-windowed.zip:   0%|          | 0.00/148M [00:00<?, ?B/s]

In [ ]:
!unzip vectorstore-windowed.zip

Archive:  vectorstore-windowed.zip
   creating: ai_tutor_knowledge/0201aaa7-f5de-4efd-8594-fd5c929d6fdf/
  inflating: ai_tutor_knowledge/0201aaa7-f5de-4efd-8594-fd5c929d6fdf/header.bin  
  inflating: ai_tutor_knowledge/0201aaa7-f5de-4efd-8594-fd5c929d6fdf/data_level0.bin  
  inflating: ai_tutor_knowledge/0201aaa7-f5de-4efd-8594-fd5c929d6fdf/link_lists.bin  
  inflating: ai_tutor_knowledge/0201aaa7-f5de-4efd-8594-fd5c929d6fdf/length.bin  
  inflating: ai_tutor_knowledge/0201aaa7-f5de-4efd-8594-fd5c929d6fdf/index_metadata.pickle  
replace ai_tutor_knowledge/chroma.sqlite3? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex

# # Create your index
db = chromadb.PersistentClient(path="./ai_tutor_knowledge")
chroma_collection = db.get_or_create_collection("ai_tutor_knowledge")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# # Create your index
vector_index = VectorStoreIndex.from_vector_store(vector_store)

In [ ]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

query_engine = vector_index.as_query_engine(
    llm=Settings.llm,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)

In [ ]:
response = query_engine.query("Write about naive RAG and Speculative RAG?")
print(response)

In [ ]:
for idx, item in enumerate(response.source_nodes):
    print("Source ", idx + 1)
    print("Original Text:", item.node.metadata["original_text"])
    print("Window:", item.node.metadata["window"])
    print("----")